In [1]:
import pandas as pd
import numpy as np
import implicit
import scipy.sparse as sparse

In [ ]:
# WARNING: ONLY RUN ONCE OR DATA WILL BE ERASED 

# Instance datasets
train_file = pd.read_csv('dataset/train.csv', sep=',', header=0)
test_file = pd.read_csv('dataset/validation.csv', sep=',', header=0)
anime_file = pd.read_csv('dataset/anime.csv', sep=',', header=0)

# # Remove the registers with rating -1
# train_file = train_file[train_file['rating'] != -1]
# test_file = test_file[test_file['rating'] != -1]

# # Remove the first row from CSV
# train_file.to_csv("dataset/train.csv", index=False, header=False)
# test_file.to_csv("dataset/validation.csv", index=False, header=False)
# anime_file.to_csv("dataset/anime.csv", index=False, header=False)

train_file = pd.read_csv('dataset/train.csv', names = ['user_id','item_id','rating'] ,sep=',', header=0)
test_file = pd.read_csv('dataset/validation.csv', names = ['user_id','item_id','rating'], sep=',', header=0)
anime_file = pd.read_csv('dataset/anime.csv', names = ['anime_id','name','genre','type','episodes','rating','members'], sep=',', header=0)

train_file.head()